# Wan2.2 Video Generation on Google Colab

Wan2.2をGoogle Colabで動かすためのノートブック

## 特徴
- **Google Driveにモデル保存** - 毎回ダウンロード不要
- **複数モデル対応** - 用途に応じて選択可能
- **GGUF量子化対応** - T4でも14Bモデル動作可能
- **カスタムノード対応** - WanVideoWrapper, GGUF, VideoHelperSuite

## 対応モデル

### Diffusion Models（モジュラー型）
| モデル | サイズ | VRAM目安 | 用途 |
|--------|--------|----------|------|
| I2V-14B-fp16 | 28.6GB | 24GB+ | 高品質I2V |
| I2V-14B-fp8 | 14.3GB | ~16GB | 中VRAM I2V |
| I2V-14B-GGUF-Q4 | ~8GB | ~12GB | T4対応I2V |
| T2V-14B-fp16 | 28.6GB | 24GB+ | 高品質T2V |
| T2V-14B-fp8 | 14.3GB | ~16GB | 中VRAM T2V |
| TI2V-5B | ~10GB | ~8GB | 軽量版（テキスト+画像） |
| Animate-14B-GGUF-Q4 | ~8GB | ~12GB | アニメーション |

### High Noise vs Low Noise
- **High Noise**: 大きな動き・変化を生成（推奨）
- **Low Noise**: 入力により忠実、変化控えめ

## ワークフロー
| ファイル | 用途 | 必要モデル |
|----------|------|------------|
| t2v-14b.json | テキスト→動画 | T2V-14B + TextEncoder + VAE |
| i2v-14b.json | 画像→動画 | I2V-14B + TextEncoder + VAE + CLIP Vision |
| low-vram-ti2v.json | T4対応TI2V | TI2V-5B + LoRA + TextEncoder + VAE |

---
## 1. GPU確認

In [ ]:
import torch

!nvidia-smi
print("\n" + "="*50)
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
    if gpu_memory >= 40:
        print("推奨: fp16モデル (I2V-14B, T2V-14B)")
    elif gpu_memory >= 20:
        print("推奨: fp16またはGGUF-Q8")
    else:
        print("推奨: GGUF-Q4量子化モデル or TI2V-5B")
else:
    print("GPU not available!")
print("="*50)

---
## 2. Google Driveマウント

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# モデル保存用ディレクトリ
DRIVE_MODEL_DIR = "/content/drive/MyDrive/ComfyUI_Wan22"

# フォルダ構成作成
folders = [
    f"{DRIVE_MODEL_DIR}/diffusion_models",
    f"{DRIVE_MODEL_DIR}/text_encoders",
    f"{DRIVE_MODEL_DIR}/vae",
    f"{DRIVE_MODEL_DIR}/clip_vision",
    f"{DRIVE_MODEL_DIR}/loras",
    f"{DRIVE_MODEL_DIR}/workflows",
    f"{DRIVE_MODEL_DIR}/outputs"
]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print(f"モデル保存先: {DRIVE_MODEL_DIR}")

# GitHubからワークフローをダウンロード
GITHUB_RAW = "https://raw.githubusercontent.com/hatoya0703/Colab.Wan/main/workflows"
workflows = [
    "t2v-14b.json",
    "i2v-14b.json",
    "low-vram-ti2v.json"
]
print("\nワークフローをGitHubからダウンロード:")
for wf in workflows:
    dest = f"{DRIVE_MODEL_DIR}/workflows/{wf}"
    if not os.path.exists(dest):
        !wget -q -O "{dest}" "{GITHUB_RAW}/{wf}"
        print(f"  DL: {wf}")
    else:
        print(f"  既存: {wf}")

print("\n既存のモデル:")
!ls -la "{DRIVE_MODEL_DIR}/diffusion_models/" 2>/dev/null || echo "(なし)"
!ls -la "{DRIVE_MODEL_DIR}/text_encoders/" 2>/dev/null || echo "(なし)"
print("\n既存のワークフロー:")
!ls -la "{DRIVE_MODEL_DIR}/workflows/" 2>/dev/null || echo "(なし)"

---
## 3. 設定

In [ ]:
#@title 設定パネル { display-mode: "form" }

#@markdown ### Low-VRAM TI2V（T4対応）
DL_LOW_VRAM_TI2V = False #@param {type:"boolean"}

#@markdown ### Diffusionモデル（複数選択可）
DL_I2V_14B_FP16 = False #@param {type:"boolean"}
DL_I2V_14B_FP8 = False #@param {type:"boolean"}
DL_I2V_14B_GGUF_Q4 = True #@param {type:"boolean"}
DL_ANIMATE_14B_GGUF_Q4 = False #@param {type:"boolean"}
DL_TI2V_5B = False #@param {type:"boolean"}
DL_T2V_14B_FP16 = False #@param {type:"boolean"}
DL_T2V_14B_FP8 = False #@param {type:"boolean"}

#@markdown ### 共通モデル（Diffusion Models用）
DL_TEXT_ENCODER = True #@param {type:"boolean"}
DL_VAE = True #@param {type:"boolean"}
DL_CLIP_VISION = True #@param {type:"boolean"}
DL_LORA_ANIMATE = False #@param {type:"boolean"}

# 選択されたモデルを表示
print("=== ダウンロード対象 ===")
selected = []
if DL_LOW_VRAM_TI2V: selected.append("Low-VRAM TI2V (5B + LoRA)")
if DL_I2V_14B_FP16: selected.append("I2V-14B-fp16 (High+Low Noise)")
if DL_I2V_14B_FP8: selected.append("I2V-14B-fp8 (High Noise)")
if DL_I2V_14B_GGUF_Q4: selected.append("I2V-14B-GGUF-Q4")
if DL_ANIMATE_14B_GGUF_Q4: selected.append("Animate-14B-GGUF-Q4")
if DL_TI2V_5B: selected.append("TI2V-5B")
if DL_T2V_14B_FP16: selected.append("T2V-14B-fp16 (High+Low Noise)")
if DL_T2V_14B_FP8: selected.append("T2V-14B-fp8 (High Noise)")
if DL_TEXT_ENCODER: selected.append("Text Encoder (umt5-xxl-fp8)")
if DL_VAE: selected.append("VAE")
if DL_CLIP_VISION: selected.append("CLIP Vision")
if DL_LORA_ANIMATE: selected.append("LoRA (WanAnimate relight)")

for s in selected:
    print(f"  - {s}")
if not selected:
    print("  (なし - 既存モデルを使用)")

# Low-VRAM TI2V使用時の注意
if DL_LOW_VRAM_TI2V:
    print("\n💡 Low-VRAM TI2V選択時:")
    print("  → T4 GPU対応（~12GB VRAM）")
    print("  → TI2V-5Bモデル + PusaV1 LoRA使用")
    print("  → SageAttention有効化推奨")

---
## 4. モデルダウンロード（初回 or 追加時のみ実行）

既にGoogle Driveにモデルがある場合はスキップ可能

In [ ]:
%%time
from huggingface_hub import hf_hub_download
import os
import shutil

def download_model(repo_id, filename, dest_folder, dest_filename=None):
    """モデルをダウンロードしてGoogle Driveに保存"""
    dest_path = f"{DRIVE_MODEL_DIR}/{dest_folder}/{dest_filename or os.path.basename(filename)}"
    if os.path.exists(dest_path):
        print(f"  既存: {os.path.basename(dest_path)}")
        return
    print(f"  DL中: {filename}")
    downloaded = hf_hub_download(repo_id=repo_id, filename=filename)
    shutil.copy(downloaded, dest_path)
    print(f"  保存: {dest_path}")

# === Low-VRAM TI2V ===
if DL_LOW_VRAM_TI2V:
    print("\n=== Low-VRAM TI2V ===")
    print("\n[TI2V-5B Diffusion Model]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_repackaged",
        "split_files/diffusion_models/wan2.2_ti2v_5B_fp16.safetensors",
        "diffusion_models"
    )
    print("\n[Text Encoder (scaled)]")
    download_model(
        "Comfy-Org/Wan_2.1_ComfyUI_repackaged",
        "split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors",
        "text_encoders"
    )
    print("\n[VAE (wan2.2)]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_Repackaged",
        "split_files/vae/wan2.2_vae.safetensors",
        "vae"
    )
    print("\n[PusaV1 LoRA]")
    download_model(
        "Kijai/WanVideo_comfy",
        "Pusa/Wan21_PusaV1_LoRA_14B_rank512_bf16.safetensors",
        "loras",
        "Wan21_PusaV1_LoRA_14B_rank512_bf16.safetensors"
    )
    print("\n[CLIP Vision]")
    download_model(
        "Comfy-Org/Wan_2.1_ComfyUI_repackaged",
        "split_files/clip_vision/clip_vision_h.safetensors",
        "clip_vision"
    )

# === Diffusion Models ===
print("\n=== Diffusion Models ===")

if DL_I2V_14B_FP16:
    print("\n[I2V-14B-fp16 High Noise]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_Repackaged",
        "split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp16.safetensors",
        "diffusion_models"
    )
    print("\n[I2V-14B-fp16 Low Noise]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_Repackaged",
        "split_files/diffusion_models/wan2.2_i2v_low_noise_14B_fp16.safetensors",
        "diffusion_models"
    )

if DL_I2V_14B_FP8:
    print("\n[I2V-14B-fp8 High Noise]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_Repackaged",
        "split_files/diffusion_models/wan2.2_i2v_high_noise_14B_fp8_scaled.safetensors",
        "diffusion_models"
    )

if DL_I2V_14B_GGUF_Q4:
    print("\n[I2V-14B-GGUF-Q4]")
    download_model(
        "city96/Wan2.1-I2V-14B-480P-GGUF",
        "wan2.1-i2v-14b-480p-Q4_K_S.gguf",
        "diffusion_models"
    )

if DL_ANIMATE_14B_GGUF_Q4:
    print("\n[Animate-14B-GGUF-Q4]")
    download_model(
        "Kijai/WanVideo_comfy_GGUF",
        "Wan2_2_Animate_14B_Q4_K_M.gguf",
        "diffusion_models"
    )

if DL_TI2V_5B:
    print("\n[TI2V-5B]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_repackaged",
        "split_files/diffusion_models/wan2.2_ti2v_5B_fp16.safetensors",
        "diffusion_models"
    )

if DL_T2V_14B_FP16:
    print("\n[T2V-14B-fp16 High Noise]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_Repackaged",
        "split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp16.safetensors",
        "diffusion_models"
    )
    print("\n[T2V-14B-fp16 Low Noise]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_Repackaged",
        "split_files/diffusion_models/wan2.2_t2v_low_noise_14B_fp16.safetensors",
        "diffusion_models"
    )

if DL_T2V_14B_FP8:
    print("\n[T2V-14B-fp8 High Noise]")
    download_model(
        "Comfy-Org/Wan_2.2_ComfyUI_Repackaged",
        "split_files/diffusion_models/wan2.2_t2v_high_noise_14B_fp8_scaled.safetensors",
        "diffusion_models"
    )

# === Text Encoder ===
if DL_TEXT_ENCODER:
    print("\n=== Text Encoder ===")
    download_model(
        "Kijai/WanVideo_comfy",
        "umt5-xxl-enc-fp8_e4m3fn.safetensors",
        "text_encoders"
    )

# === VAE ===
if DL_VAE:
    print("\n=== VAE ===")
    download_model(
        "Kijai/WanVideo_comfy",
        "Wan2_1_VAE_bf16.safetensors",
        "vae"
    )

# === CLIP Vision ===
if DL_CLIP_VISION:
    print("\n=== CLIP Vision ===")
    download_model(
        "Comfy-Org/Wan_2.1_ComfyUI_repackaged",
        "split_files/clip_vision/clip_vision_h.safetensors",
        "clip_vision"
    )

# === LoRA ===
if DL_LORA_ANIMATE:
    print("\n=== LoRA ===")
    download_model(
        "Kijai/WanVideo_comfy",
        "Wan22_relight/WanAnimate_relight_lora_fp16.safetensors",
        "loras",
        "WanAnimate_relight_lora_fp16.safetensors"
    )

print("\n" + "="*50)
print("ダウンロード完了!")
print("="*50)
print("\n保存されたモデル:")
!ls -lh "{DRIVE_MODEL_DIR}/diffusion_models/"
!ls -lh "{DRIVE_MODEL_DIR}/text_encoders/"
!ls -lh "{DRIVE_MODEL_DIR}/vae/"
!ls -lh "{DRIVE_MODEL_DIR}/loras/" 2>/dev/null || echo "(なし)"

---
## 5. ComfyUI + カスタムノード インストール

In [ ]:
%%time
import os

# ComfyUI
if not os.path.exists("/content/ComfyUI"):
    !git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt
!pip install -q huggingface_hub
print("ComfyUI installed.")

# Kijai WanVideoWrapper
if not os.path.exists("custom_nodes/ComfyUI-WanVideoWrapper"):
    !git clone https://github.com/kijai/ComfyUI-WanVideoWrapper.git custom_nodes/ComfyUI-WanVideoWrapper
    !pip install -q -r custom_nodes/ComfyUI-WanVideoWrapper/requirements.txt
print("WanVideoWrapper installed.")

# GGUF support
if not os.path.exists("custom_nodes/ComfyUI-GGUF"):
    !git clone https://github.com/city96/ComfyUI-GGUF.git custom_nodes/ComfyUI-GGUF
    !pip install -q gguf
print("GGUF support installed.")

# VideoHelperSuite
if not os.path.exists("custom_nodes/ComfyUI-VideoHelperSuite"):
    !git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git custom_nodes/ComfyUI-VideoHelperSuite
    !pip install -q -r custom_nodes/ComfyUI-VideoHelperSuite/requirements.txt 2>/dev/null || true
print("VideoHelperSuite installed.")

# KJNodes (for Low-VRAM TI2V workflow)
if not os.path.exists("custom_nodes/ComfyUI-KJNodes"):
    !git clone https://github.com/kijai/ComfyUI-KJNodes.git custom_nodes/ComfyUI-KJNodes
    !pip install -q -r custom_nodes/ComfyUI-KJNodes/requirements.txt 2>/dev/null || true
print("KJNodes installed.")

# rgthree-comfy (for Low-VRAM TI2V workflow)
if not os.path.exists("custom_nodes/rgthree-comfy"):
    !git clone https://github.com/rgthree/rgthree-comfy.git custom_nodes/rgthree-comfy
print("rgthree-comfy installed.")

# SageAttention (for Low-VRAM optimization)
!pip install -q sageattention
print("SageAttention installed.")

print("\nAll custom nodes installed!")

---
## 6. モデルをシンボリックリンクで接続

In [ ]:
import os

os.chdir("/content/ComfyUI")

# 既存のmodelsフォルダを削除
!rm -rf models/diffusion_models models/text_encoders models/vae models/clip_vision models/loras 2>/dev/null

# シンボリックリンク作成（モデル）
links = [
    (f"{DRIVE_MODEL_DIR}/diffusion_models", "models/diffusion_models"),
    (f"{DRIVE_MODEL_DIR}/text_encoders", "models/text_encoders"),
    (f"{DRIVE_MODEL_DIR}/vae", "models/vae"),
    (f"{DRIVE_MODEL_DIR}/clip_vision", "models/clip_vision"),
    (f"{DRIVE_MODEL_DIR}/loras", "models/loras"),
]

for src, dst in links:
    if os.path.exists(src):
        !ln -sf "{src}" "{dst}"
        print(f"リンク: {dst} -> {src}")

# ワークフローのシンボリックリンク
os.makedirs("user/default", exist_ok=True)
!rm -rf user/default/workflows 2>/dev/null
if os.path.exists(f"{DRIVE_MODEL_DIR}/workflows"):
    !ln -sf "{DRIVE_MODEL_DIR}/workflows" "user/default/workflows"
    print(f"リンク: user/default/workflows -> {DRIVE_MODEL_DIR}/workflows")

print("\n=== モデル確認 ===")
!ls -la models/diffusion_models/
!ls -la models/text_encoders/
!ls -la models/vae/
!ls -la models/clip_vision/
print("\n=== ワークフロー確認 ===")
!ls -la user/default/workflows/ 2>/dev/null || echo "workflows: (なし)"

---
## 7. ComfyUI起動

In [ ]:
import os
os.chdir("/content/ComfyUI")

# Colab SecretsからNGROKトークンを取得
from google.colab import userdata
try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
except userdata.SecretNotFoundError:
    NGROK_TOKEN = None
    print("⚠️ NGROK_TOKEN が Colab Secrets に設定されていません")
    print("  左サイドバー 🔑 → 新しいシークレットを追加 → NGROK_TOKEN")

# ngrokトンネル
if NGROK_TOKEN:
    !pip install -q pyngrok
    from pyngrok import ngrok
    ngrok.kill()  # 既存のトンネルを停止
    ngrok.set_auth_token(NGROK_TOKEN)
    public_url = ngrok.connect(8188)
    print(f"\n{'='*50}")
    print(f"ComfyUI URL: {public_url}")
    print(f"{'='*50}\n")
else:
    print("="*50)
    print("ngrok未設定")
    print("Colabの出力ポートからアクセスしてください")
    print("="*50)

# ComfyUI起動
!python main.py --listen 0.0.0.0 --port 8188

---
## Tips

### 2回目以降の起動
1. セル1 (GPU確認)
2. セル2 (Google Driveマウント)
3. セル5 (ComfyUIインストール)
4. セル6 (シンボリックリンク)
5. セル7 (ComfyUI起動)

→ セル3,4はスキップ可能

### GPU別推奨モデル
| GPU | VRAM | 推奨モデル |
|-----|------|------------|
| T4 | 15GB | GGUF-Q4, TI2V-5B, fp8 |
| L4 | 24GB | fp16, fp8 |
| A100 | 40GB | fp16 |

### フォルダ構成
```
Google Drive/MyDrive/ComfyUI_Wan22/
├── diffusion_models/  # メインモデル
├── text_encoders/     # テキストエンコーダー
├── vae/               # VAE
├── clip_vision/       # CLIP Vision
├── loras/             # LoRA
├── workflows/         # ワークフロー（自動読み込み）
└── outputs/           # 出力動画
```

### 14B推奨設定
- Sampler: `uni_pc`
- Scheduler: `simple`
- CFG: `5`
- Steps: `30`
- ModelSamplingSD3 shift: `3.0`